In [1]:
moons1 = """<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>"""
moons2 = """<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>"""
moons_p ="""<x=-7, y=17, z=-11>
<x=9, y=12, z=5>
<x=-9, y=0, z=-4>
<x=4, y=6, z=0>"""

In [2]:
from tqdm import tqdm_notebook as tqdm

In [2]:
def convert(moons):
    tab = str.maketrans('','',"<>xyz=")
    pos = moons.translate(tab)
    pos = [eval(f"[{l}]") for l in pos.splitlines()]
    
    vel = [[0,0,0] for i in range(len(pos))]
    
    return pos,vel

In [3]:
def ggt(a,b):
    a,b = abs(a), abs(b)
    while b!=0:
        a, b = b, a % b
    return a

In [5]:
def run(moons, N, i=None):
    pos, vel = convert(moons)
    
    if i:
        pos = [[pos[n][i]] for n in range(len(pos))]
        vel = [[vel[n][i]] for n in range(len(pos))]
    
    def update():
        dims = len(pos[0])
        for m1 in range(len(pos)):
            for m2 in range(len(pos)):
                if m1==m2: continue
                for i in range(dims):
                    c1, c2 = pos[m1][i], pos[m2][i]
                    if   c1 > c2: vel[m1][i] -= 1
                    elif c1 < c2: vel[m1][i] += 1
        for m in range(len(pos)):
            for i in range(dims):
                pos[m][i] += vel[m][i]
        #print(f"pos={pos}, vel={vel}")
    
    if N != None:
        for step in range(N):
            update()
    
        energy = sum([
            sum(map(abs,pos[i])) * sum(map(abs,vel[i])) for i in range(len(pos))
        ])
        return energy
    
    else:
        path = set([str([pos, vel])])
        step = 0
        with tqdm() as t:
            t.set_description('i = %i' % i)
            while True:
                update()
                t.update()
                step += 1
                if str([pos, vel]) in path:
                    #print([pos, vel])
                    break
                path.add(str([pos, vel]))
            return step

In [6]:
run(moons1, 10)

179

In [7]:
run(moons2, 100)

1940

In [8]:
run(moons_p, 1000)

7013

In [9]:
res = [run(moons1, None, i) for i in range(3)]
l = res[0]*res[1]//ggt(res[0],res[1])
l = l*res[2]//ggt(l, res[2])
print(res, l)


[2772, 28, 44] 2772


In [13]:
res = [run(moons2, None, i) for i in range(3)]
l = res[0]*res[1]//ggt(res[0],res[1])
l = l*res[2]//ggt(l, res[2])
print(res, l)

KeyboardInterrupt: 

# New attempt

In [29]:
from numba import jit
import numpy as np

In [46]:
def convert2(moons, i):
    tab = str.maketrans('','',"<>xyz=")
    pos = moons.translate(tab)
    pos = [eval(f"[{l}]") for l in pos.splitlines()]
    return [pos[n][i] for n in range(len(pos))]

In [47]:
def kgv(a,b):
    return a*b//ggt(a,b)

In [55]:
@jit(nopython=True)
def run_fast(pos):
    vel = np.zeros(4, dtype=np.int64)
    steps = 0
    while True:
        steps += 1
        
        for m1 in range(4):
            for m2 in range(4):
                if m1==m2: continue
                c1, c2 = pos[m1], pos[m2]
                if   c1 > c2: vel[m1] -= 1
                elif c1 < c2: vel[m1] += 1
        
        for m in range(4):
            pos[m] += vel[m]
            
        if not np.any(vel):
            return 2 * steps

In [54]:
np.zeros(4, dtype=np.int64)

array([0, 0, 0, 0])

In [59]:
%timeit a,b,c = (run_fast( np.array(convert2(moons_p, i), dtype=np.int64) ) for i in range(3))
print(kgv(a,kgv(b,c)))

4.32 ms ± 120 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
324618307124784
